<a href="https://colab.research.google.com/github/prahlad-pallav/Stock_Market_Analysis/blob/main/LTSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install yfinance numpy pandas matplotlib scikit-learn tensorflow

In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [3]:
# 1. Load RELIANCE stock data
df = yf.download("RELIANCE.NS", start="2018-01-01", end="2024-01-01")
data = df[['Close']].values

/tmp/ipython-input-3-3753428227.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download("RELIANCE.NS", start="2018-01-01", end="2024-01-01")
[*********************100%***********************]  1 of 1 completed


In [4]:
# 2. Normalize the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

In [6]:
# 3. Create sequences for training
def create_sequences(data, window):
    X, y = [], []
    for i in range(window, len(data)):
        X.append(data[i-window:i])
        y.append(data[i])
    return np.array(X), np.array(y)

window_size = 60
X, y = create_sequences(scaled_data, window_size)

In [7]:
# 4. Reshape for LSTM input
X = X.reshape((X.shape[0], X.shape[1], 1))

In [8]:
# 5. Build LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(window_size, 1)))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
# 6. Train the model
model.fit(X, y, epochs=10, batch_size=32, verbose=1)

Epoch 1/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 6s 47ms/step - loss: 0.0904
Epoch 2/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - loss: 0.0020
Epoch 3/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - loss: 0.0019
Epoch 4/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - loss: 0.0017
Epoch 5/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 70ms/step - loss: 0.0018
Epoch 6/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - loss: 0.0014
Epoch 7/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - loss: 0.0015
Epoch 8/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - loss: 0.0014
Epoch 9/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - loss: 0.0014
Epoch 10/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - loss: 0.0012


In [10]:
# 7. Predict the next day's price
last_sequence = scaled_data[-window_size:]
pred_input = last_sequence.reshape(1, window_size, 1)
predicted_price = model.predict(pred_input)
predicted_price = scaler.inverse_transform(predicted_price)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [11]:
print(f"📈 Predicted next closing price for RELIANCE: ₹{predicted_price[0][0]:.2f}")

📈 Predicted next closing price for RELIANCE: ₹1285.50
